In [1]:
import pandas as pd

# 먼저 utf-8로 시도
try:
    df = pd.read_csv('seoul_20240301_final.csv', encoding='utf-8')
except UnicodeDecodeError:
    # 깨지면 cp949 (또는 euc-kr)로 다시 시도
    df = pd.read_csv('seoul_20240301_final.csv', encoding='cp949')


In [5]:
df.head(5)

,출발 행정동,도착 행정동,출발 시간,도착 시간,평균 이동거리(m),평균 이동시간(분),0~19세,20~39세,40~59세,60세 이상,전체 인구수,기준 년월일,출발 시도명,출발 시군구명,출발 행정동명,도착 시도명,도착 시군구명,도착 행정동명
0,11110515.0,11110515.0,0,0,517.76,2.633833,0.0,12.35,13.15,0.0,25.49,20240301,서울특별시,종로구,청운효자동,서울특별시,종로구,청운효자동
1,11110515.0,11110530.0,0,0,919.20,10.405000,0.0,2.72,3.29,0.0,6.01,20240301,서울특별시,종로구,청운효자동,서울특별시,종로구,사직동
2,11110515.0,11110540.0,0,0,360.60,2.450000,0.0,4.50,0.00,0.0,4.50,20240301,서울특별시,종로구,청운효자동,서울특별시,종로구,삼청동
3,11110515.0,11110550.0,0,0,1588.90,11.371389,0.0,7.01,2.14,0.0,9.15,20240301,서울특별시,종로구,청운효자동,서울특별시,종로구,부암동
4,11110515.0,11440555.0,0,0,2828.85,11.024167,0.0,3.94,0.00,0.0,3.94,20240301,서울특별시,종로구,청운효자동,서울특별시,마포구,아현동


In [2]:
from google.colab import drive
import pandas as pd

# 🔷 구글 드라이브 마운트
drive.mount('/content/drive')

# 🔷 함수 정의
def process_seoul_movement(input_csv_path: str, admi_csv_path: str, output_path: str):
    """
    서울 생활이동 데이터셋을 가공해 최종본(csv)을 저장합니다.
    """
    print(f"📂 처리 시작: {input_csv_path}")

    df = pd.read_csv(input_csv_path, dtype=str)
    admi_df = pd.read_csv(admi_csv_path, dtype=str)

    o_mapping = admi_df.rename(columns={
        'SIDO_NM': 'O_SIDO_NM',
        'SGG_NM': 'O_SGG_NM',
        'ADMI_NM': 'O_ADMI_NM',
        'ADMI_CD': 'O_ADMDONG_CD'
    })[['O_ADMDONG_CD', 'O_SIDO_NM', 'O_SGG_NM', 'O_ADMI_NM']]

    d_mapping = admi_df.rename(columns={
        'SIDO_NM': 'D_SIDO_NM',
        'SGG_NM': 'D_SGG_NM',
        'ADMI_NM': 'D_ADMI_NM',
        'ADMI_CD': 'D_ADMDONG_CD'
    })[['D_ADMDONG_CD', 'D_SIDO_NM', 'D_SGG_NM', 'D_ADMI_NM']]

    df['O_ADMDONG_CD'] = df['O_ADMDONG_CD'].astype(str)
    df['D_ADMDONG_CD'] = df['D_ADMDONG_CD'].astype(str)

    df = df.merge(o_mapping, on='O_ADMDONG_CD', how='left')
    df = df.merge(d_mapping, on='D_ADMDONG_CD', how='left')

    col_map = {
        'O_ADMDONG_CD': '출발 행정동',
        'D_ADMDONG_CD': '도착 행정동',
        'ST_TIME_CD': '출발 시간',
        'FNS_TIME_CD': '도착 시간',
        'MOVE_DIST': '평균 이동거리(m)',
        'MOVE_TIME': '평균 이동시간(분)',
        '0010_CNT': '0~19세',
        '2030_CNT': '20~39세',
        '4050_CNT': '40~59세',
        '60PLUS_CNT': '60세 이상',
        'TOTAL_CNT': '전체 인구수',
        'ETL_YMD': '기준 년월일',
        'O_SIDO_NM': '출발 시도명',
        'O_SGG_NM': '출발 시군구명',
        'O_ADMI_NM': '출발 행정동명',
        'D_SIDO_NM': '도착 시도명',
        'D_SGG_NM': '도착 시군구명',
        'D_ADMI_NM': '도착 행정동명'
    }
    df.rename(columns=col_map, inplace=True)

    numeric_cols = [
        '출발 행정동', '도착 행정동', '출발 시간', '도착 시간',
        '평균 이동거리(m)', '평균 이동시간(분)',
        '0~19세', '20~39세', '40~59세', '60세 이상', '전체 인구수'
    ]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    df.to_csv(output_path, index=False)
    print(f"✅ 최종 저장 완료: {output_path}\n")

# 🔷 admi 경로
admi_csv_path = "/content/drive/MyDrive/수도권 생활이동 데이터 해커톤/ADMI_202403.csv"

# 🔷 처리할 날짜 목록 (6일 제외)
dates = ['03', '04', '05', '07']

for day in dates:
    input_csv_path = f"/content/drive/MyDrive/수도권 생활이동 데이터 해커톤/서울 열린데이터 광장/2024_03/seoul_purpose_admdong4_in_202403{day}.csv"
    output_path = f"/content/drive/MyDrive/수도권 생활이동 데이터 해커톤/서울 열린데이터 광장/2024_03/seoul_202403{day}_final.csv"
    process_seoul_movement(input_csv_path, admi_csv_path, output_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 처리 시작: /content/drive/MyDrive/수도권 생활이동 데이터 해커톤/서울 열린데이터 광장/2024_03/seoul_purpose_admdong4_in_20240303.csv
✅ 최종 저장 완료: /content/drive/MyDrive/수도권 생활이동 데이터 해커톤/서울 열린데이터 광장/2024_03/seoul_20240303_final.csv

📂 처리 시작: /content/drive/MyDrive/수도권 생활이동 데이터 해커톤/서울 열린데이터 광장/2024_03/seoul_purpose_admdong4_in_20240304.csv
✅ 최종 저장 완료: /content/drive/MyDrive/수도권 생활이동 데이터 해커톤/서울 열린데이터 광장/2024_03/seoul_20240304_final.csv

📂 처리 시작: /content/drive/MyDrive/수도권 생활이동 데이터 해커톤/서울 열린데이터 광장/2024_03/seoul_purpose_admdong4_in_20240305.csv
✅ 최종 저장 완료: /content/drive/MyDrive/수도권 생활이동 데이터 해커톤/서울 열린데이터 광장/2024_03/seoul_20240305_final.csv

📂 처리 시작: /content/drive/MyDrive/수도권 생활이동 데이터 해커톤/서울 열린데이터 광장/2024_03/seoul_purpose_admdong4_in_20240307.csv
✅ 최종 저장 완료: /content/drive/MyDrive/수도권 생활이동 데이터 해커톤/서울 열린데이터 광장/2024_03/seoul_20240307_final.csv

